### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load
school_data_to_load = Path("../Resources/schools_complete.csv")
student_data_to_load = Path("../Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(
    student_data, school_data, how="left", on=["school_name", "school_name"])

# Clean up the column headings
complete_df = school_data_complete.rename(columns={
    "student_name": "Student Name",
    "gender": "Gender",
    "year": "Year",
    "school_name": "School Name",
    "reading_score": "Reading Score",
    "maths_score": "Maths Score",
    "type": "School Type",
    "size": "School Size",
    "budget": "School Budget"})

# Rearrange columns for readability
complete_df = complete_df[[
    "Student ID",
    "Student Name",
    "Gender",
    "Year",
    "Reading Score",
    "Maths Score",
    "School ID",
    "School Name",
    "School Type",
    "School Size",
    "School Budget"]]

complete_df.head()

,Student ID,Student Name,Gender,Year,Reading Score,Maths Score,School ID,School Name,School Type,School Size,School Budget
0,0,Paul Bradley,M,9,96,94,0,Huang High School,Government,2917,1910635
1,1,Victor Smith,M,12,90,43,0,Huang High School,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,41,76,0,Huang High School,Government,2917,1910635
3,3,Richard Scott,M,12,89,86,0,Huang High School,Government,2917,1910635
4,4,Bonnie Ray,F,9,87,69,0,Huang High School,Government,2917,1910635


## Local Government Area Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average maths score 

* Calculate the average reading score

* Calculate the percentage of students with a passing maths score (50 or greater)

* Calculate the percentage of students with a passing reading score (50 or greater)

* Calculate the percentage of students who passed maths **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [7]:
# Calculate the total number of schools
total_schools = len(complete_df["School ID"].unique())

# Calculate the total number of students
total_students = len(complete_df["Student ID"].unique())

# Calculate the total budget
total_budget = complete_df["School Budget"].unique().sum()

# Calculate the average maths score
average_maths = complete_df["Maths Score"].mean()

# Calculate the average reading score
average_reading = complete_df["Reading Score"].mean()

# Calculate the percentage of students with a passing maths score (>= 50)
passing_score = 50
num_passers_math = complete_df.loc[
    complete_df["Maths Score"] >= passing_score, "Maths Score"].count()
percent_passed_maths = 100 * (num_passers_math / total_students)

# Calculate the percentage of students with a passing reading score (>= 50)
num_passers_read = complete_df.loc[
    complete_df["Reading Score"] >= passing_score, "Reading Score"].count()
percent_passed_reading = 100 * (num_passers_read / total_students)

# Calculate the percentage of students who passed maths AND reading
pass_math = complete_df["Maths Score"] >= passing_score
pass_read = complete_df["Reading Score"] >= passing_score
num_passers = complete_df.loc[pass_math & pass_read, "Student ID"].count()
percent_passers = 100 * (num_passers / total_students)

# Create a dataframe called area_summary
area_summary_data = [{
    "Total Schools": total_schools,
    "Total Students": total_students,
    "Total Budget": total_budget,
    "Average Maths Score": average_maths,
    "Average Reading Score": average_reading,
    "% Passing Maths": percent_passed_maths,
    "% Passing Reading": percent_passed_reading,
    "% Overall Passing": percent_passers}]
area_summary = pd.DataFrame(area_summary_data)

# Format dataframe with correct units
area_summary.style.format({
    "Total Students": "{0:,.0f}",
    "Total Budget": "${0:,.2f}"})

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


## School Summary

* Create an overview table that summarises key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed maths **and** reading.)
  
* Create a dataframe to hold the above results

In [30]:
schools_list = complete_df["School Name"].unique()

school_type = []
students_per_school = []
school_budget = []
budget_per_student = []
school_average_maths = []
school_average_reading = []
school_passed_math = []
school_passed_read = []
school_passed_both = []

for index, school in enumerate(schools_list):
    # Check school name
    school_name = (complete_df["School Name"] == school)
    
    # Get the school type for each school
    school_type.append(
        complete_df.loc[school_name, "School Type"].unique()[0])
    
    # Get total students per school
    students_per_school.append(
        complete_df.loc[school_name, "Student ID"].count())
    
    # Get the total budget per school
    school_budget.append(
        complete_df.loc[school_name, "School Budget"].unique()[0])
    
    # Get the school budget per student
    budget_per_student.append(
        school_budget[index] / students_per_school[index])
    
    # Get average maths score per school
    school_average_maths.append(
        complete_df.loc[school_name, "Maths Score"].mean())
    
    # Get average reading score per school
    school_average_reading.append(
        complete_df.loc[school_name, "Reading Score"].mean())
    
    # Get percentage of students who passed maths
    passed_maths_conditions = ((school_name) & (complete_df["Maths Score"] >= passing_score))
    num_maths_per_school = complete_df[passed_maths_conditions]["Student ID"].count()
    school_passed_math.append(100 * num_maths_per_school / students_per_school[index])
    
    # Get percentage of students who passed reading
    passed_reading_conditions = (school_name) & (complete_df["Reading Score"] >= passing_score)
    num_reading_per_school = complete_df[passed_reading_conditions]["Student ID"].count()
    school_passed_read.append(100 * num_reading_per_school / students_per_school[index])
    
    # Get percentage overall passing
    passed_both_conditions = ((school_name)
                              & (complete_df["Maths Score"] >= passing_score)
                              & (complete_df["Reading Score"] >= 50))
    num_both_per_school = complete_df[passed_both_conditions]["Student ID"].count()
    school_passed_both.append(
        100 * num_both_per_school / students_per_school[index])
    
# Create dictionary
school_summary_dict = {
    "School Type": school_type,
    "Total Students": students_per_school,
    "Total School Budget": school_budget,
    "Per Student Budget": budget_per_student,
    "Average Maths Score": school_average_maths,
    "Average Reading Score": school_average_reading,
    "% Passing Maths": school_passed_math,
    "% Passing Reading": school_passed_read,
    "% Overall Passing": school_passed_both}

# Create dataframe called per_school_summary
per_school_summary = pd.DataFrame(school_summary_dict, index=schools_list).sort_index()

# Format dataframe with correct units
per_school_summary.style.format({
    "Total School Budget": "${0:,.2f}",
    "Per Student Budget": "${0:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Pena High School,Independent,962,"$585,858.00",$609.00,72.088358,71.613306,91.683992,86.590437,79.209979


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [38]:
# Sort the top five performing schools by % overall passing
top_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=False).head(5)

# Format and display the results
top_schools.style.format({
    "Total School Budget": "${0:,.2f}",
    "Per Student Budget": "${0:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [39]:
# Sort the top worst performing schools by % overall passing
bottom_schools = per_school_summary.sort_values(by="% Overall Passing", ascending=True).head(5)

# Format and display the results
bottom_schools.style.format({
    "Total School Budget": "${0:,.2f}",
    "Per Student Budget": "${0:,.2f}"})

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


## Maths Scores by Year

* Create a table that lists the average maths score for students of each year level (9, 10, 11, 12) at each school.

  * Create a pandas series for each year. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [46]:
# Declare lists, to use as pandas series data later
year9_maths = []
year10_maths = []
year11_maths = []
year12_maths = []

# Populate the list by looping over the school name and year level
for school in schools_list:
    for year in range(9, 13):
        conditions = (complete_df["School Name"] == school) & (complete_df["Year"] == year)
        if (year == 9):
            year9_maths.append(complete_df.loc[conditions, "Maths Score"].mean())
        elif (year == 10):
            year10_maths.append(complete_df.loc[conditions, "Maths Score"].mean())
        elif (year == 11):
            year11_maths.append(complete_df.loc[conditions, "Maths Score"].mean())
        elif (year == 12):
            year12_maths.append(complete_df.loc[conditions, "Maths Score"].mean())

# Create a pandas series for each year, grouping by school
y9_math = pd.Series(year9_maths, name="Year 9").groupby(schools_list).mean()
y10_math = pd.Series(year10_maths, name="Year 10").groupby(schools_list).mean()
y11_math = pd.Series(year11_maths, name="Year 11").groupby(schools_list).mean()
y12_math = pd.Series(year12_maths, name="Year 12").groupby(schools_list).mean()

# Combine the series into a dataframe
maths_scores_by_year = pd.concat([y9_math, y10_math, y11_math, y12_math], axis=1)

# Display the results
maths_scores_by_year

,Year 9,Year 10,Year 11,Year 12
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393
Pena High School,71.996364,72.396000,72.523438,71.187845


# Reading Score by Year

* Perform the same operations as above for reading scores

In [47]:
# Declare lists, to use as pandas series data later
year9_reading = []
year10_reading = []
year11_reading = []
year12_reading = []

# Populate the list by looping over the school name and year level
for school in schools_list:
    for year in range(9, 13):
        conditions = (complete_df["School Name"] == school) & (complete_df["Year"] == year)
        if (year == 9):
            year9_reading.append(complete_df.loc[conditions, "Reading Score"].mean())
        elif (year == 10):
            year10_reading.append(complete_df.loc[conditions, "Reading Score"].mean())
        elif (year == 11):
            year11_reading.append(complete_df.loc[conditions, "Reading Score"].mean())
        elif (year == 12):
            year12_reading.append(complete_df.loc[conditions, "Reading Score"].mean())

# Create a pandas series for each year, grouping by school
y9_read = pd.Series(year9_reading, name="Year 9").groupby(schools_list).mean()
y10_read = pd.Series(year10_reading, name="Year 10").groupby(schools_list).mean()
y11_read = pd.Series(year11_reading, name="Year 11").groupby(schools_list).mean()
y12_read = pd.Series(year12_reading, name="Year 12").groupby(schools_list).mean()

# Combine the series into a dataframe
reading_scores_by_year = pd.concat([y9_read, y10_read, y11_read, y12_read], axis=1)

# Display the results
reading_scores_by_year

,Year 9,Year 10,Year 11,Year 12
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521
Pena High School,70.949091,72.324000,71.703125,71.513812


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Maths Score
  * Average Reading Score
  * % Passing Maths
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [109]:
# Provided code - to create bins and apply labels
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

scores_df = per_school_summary.loc[:, ["Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]

binned_budget_df = pd.cut(per_school_summary["Per Student Budget"], bins=spending_bins, labels=spending_labels)

budget_scores_df = scores_df.set_index(binned_budget_df)
budget_scores_df.index.name = "Spending Ranges (Per Student Budget)"

spending_summary = budget_scores_df.groupby(["Spending Ranges (Per Student Budget)"]).mean()
spending_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student Budget),,,,,
<$585,71.364587,70.716577,88.835926,86.390517,76.721458
$585-630,72.065868,71.031297,91.518824,87.292423,79.876293
$630-645,69.854807,69.838814,84.686139,83.763585,71.004977
$645-680,68.884391,69.045403,81.568470,81.769716,66.756253


## Scores by School Size

* Perform the same operations as above, based on school size.

In [112]:
# Provided code - to create bins and apply labels
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

binned_size_df = pd.cut(per_school_summary["Total Students"], bins=size_bins, labels=size_labels)

size_scores_df = scores_df.set_index(binned_size_df)
size_scores_df.index.name = "School Size"

size_summary = size_scores_df.groupby(["School Size"]).mean()
size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


## Scores by School Type

* Perform the same operations as above, based on school type

In [123]:
# type_summary = per_school_summary.groupby(["School Type"]).mean()
# type_summary

sliced_df = per_school_summary.loc[:, ["School Type", "Average Maths Score", "Average Reading Score", "% Passing Maths", "% Passing Reading", "% Overall Passing"]]
type_summary = sliced_df.groupby(["School Type"]).mean()
type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340
